In [ ]:
# import necassary files
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import random
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Read datasets to pandas Dataframe

In [ ]:
# product dataset
products = pd.read_csv('/content/gdrive/MyDrive/products.csv')

In [ ]:
# Merged file
final_file = pd.read_csv('/content/gdrive/MyDrive/merged_file1.csv')
final_file['days_since_prior_order']=final_file['days_since_prior_order'].fillna(-999)

## Pick users who have ordered products more than 90 because of Memory constraint

In [ ]:
final_file.head(2)

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,product_name,aisle_id,department_id
0,2539329,1,1,2,8,-999.0,196,1,Soda,77,7
1,2539329,1,1,2,8,-999.0,14084,2,Organic Unsweetened Vanilla Almond Milk,91,16


In [ ]:
total_orders_per_user = final_file.groupby('user_id')['order_id'].nunique()

In [ ]:
total_orders_per_user.sort_values(ascending=False)

user_id
26086     99
120897    99
122476    99
31118     99
1868      99
          ..
74171      3
160560     3
160570     3
74147      3
127633     3
Name: order_id, Length: 206209, dtype: int64

In [ ]:
users_bought_for_90_times = total_orders_per_user.loc[lambda x:x>=90].index

In [ ]:
len(users_bought_for_90_times)

1905

## Data Preparation

In [ ]:
new_data = final_file[final_file.user_id.isin(users_bought_for_90_times)]

In [ ]:
new_data.product_id.nunique()

27638

In [ ]:
new_data.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,product_name,aisle_id,department_id
28176,3264342,210,1,0,12,-999.0,21903,6,Organic Baby Spinach,123,4
28177,3264342,210,1,0,12,-999.0,23909,7,2% Reduced Fat Milk,84,16
28178,3264342,210,1,0,12,-999.0,28199,3,"Clementines, Bag",123,4
28179,3264342,210,1,0,12,-999.0,33043,1,Crescent Rolls,105,13
28180,3264342,210,1,0,12,-999.0,24799,4,Vanilla Skyr Nonfat Yogurt,120,16


## Create Frequent item set using Apriori

In [ ]:
new_data1 = new_data[['order_id','product_id']]
new_data1['product_id'] = new_data1['product_id'].astype('str')

In [ ]:
data3 = new_data1.groupby('order_id')['product_id'].apply(lambda x: list(','.join(x).split(','))).reset_index(name='product_list')

In [ ]:
data3.head()

,order_id,product_list
0,15,"[19660, 21195, 7461, 2996, 32463]"
1,50,"[397, 34993, 17183, 37788, 1025, 4799, 276]"
2,79,"[4210, 4377, 39461, 13627, 13110, 12397, 4840,..."
3,92,"[21903, 24852, 21137, 49235, 35561, 1376, 2660..."
4,95,"[47766, 21137, 49683, 39928, 9337, 10121, 3763..."


In [ ]:
prod = list(data3['product_list'])
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit_transform(prod)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [ ]:
df

,1,10,100,1000,10003,10005,10006,10008,10011,10013,10016,10017,10022,10023,10025,10027,10029,1003,10030,10031,10032,10034,10036,10037,10038,1004,10040,10041,10044,10048,10049,1005,10052,10054,10057,1006,10060,10065,10068,10069,...,9939,994,9940,9941,9942,9943,9945,9948,9949,995,9952,9953,9955,9957,9959,996,9960,9963,9964,9965,9966,9969,997,9970,9972,9974,9975,9976,9979,9981,9983,9984,9985,9987,9989,999,9990,9993,9995,9996
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fal

In [ ]:
del final_file,new_data,new_data1,prod

In [ ]:
frequent_itemsets  = apriori(df, min_support=0.01, use_colnames=True) 

In [ ]:
frequent_itemsets.shape

(109, 2)

## Find association rules

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.sort_values(by='confidence',ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(19057),(13176),0.033272,0.133767,0.010176,0.305839,2.286358,0.005725,1.247885
24,(47766),(24852),0.036991,0.122402,0.011075,0.299389,2.445958,0.006547,1.252619
11,(47209),(13176),0.083662,0.133767,0.024237,0.289702,2.165727,0.013046,1.219535
9,(27966),(13176),0.055292,0.133767,0.015444,0.279319,2.088104,0.008048,1.201965
17,(27966),(21137),0.055292,0.097201,0.014099,0.254988,2.623299,0.008724,1.211791
5,(21903),(13176),0.071242,0.133767,0.016074,0.225621,1.686677,0.006544,1.118617
2,(21137),(13176),0.097201,0.133767,0.021568,0.221890,1.658784,0.008566,1.113253
19,(47209),(21137),0.083662,0.097201,0.016122,0.192706,1.982547,0.007990,1.118302
27,(27966),(47209),0.055292,0.083662,0.010284,0.185985,2.223056,0.005658,1.125702
7,(27845),(13176),0.054829,0.133767,0.010036,0.183041,1.368356,0.002702,1.060314


In [ ]:
rules.shape

(28, 9)

In [ ]:
rules['antecedents'] = rules['antecedents'].apply(lambda x: int(''.join(list(x))))
rules['consequents'] = rules['consequents'].apply(lambda x: int(''.join(list(x))))

In [ ]:
rules = pd.merge(rules,products[['product_id','product_name']],left_on=['antecedents'], right_on='product_id').drop(['antecedents','product_id'],1)
rules = rules.rename(columns={'product_name':'antecedents'})

In [ ]:
rules = pd.merge(rules,products[['product_id','product_name']],left_on=['consequents'], right_on='product_id').drop(['consequents','product_id'],1)
rules = rules.rename(columns={'product_name':'consequents'})

In [ ]:
rules.sort_values(by='confidence',ascending=False)

,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents,consequents
18,0.033272,0.133767,0.010176,0.305839,2.286358,0.005725,1.247885,Organic Large Extra Fancy Fuji Apple,Bag of Organic Bananas
26,0.036991,0.122402,0.011075,0.299389,2.445958,0.006547,1.252619,Organic Avocado,Banana
23,0.083662,0.133767,0.024237,0.289702,2.165727,0.013046,1.219535,Organic Hass Avocado,Bag of Organic Bananas
22,0.055292,0.133767,0.015444,0.279319,2.088104,0.008048,1.201965,Organic Raspberries,Bag of Organic Bananas
3,0.055292,0.097201,0.014099,0.254988,2.623299,0.008724,1.211791,Organic Raspberries,Organic Strawberries
20,0.071242,0.133767,0.016074,0.225621,1.686677,0.006544,1.118617,Organic Baby Spinach,Bag of Organic Bananas
19,0.097201,0.133767,0.021568,0.221890,1.658784,0.008566,1.113253,Organic Strawberries,Bag of Organic Bananas
4,0.083662,0.097201,0.016122,0.192706,1.982547,0.007990,1.118302,Organic Hass Avocado,Organic Strawberries
17,0.055292,0.083662,0.010284,0.185985,2.223056,0.005658,1.125702,Organic Raspberries,Organic Hass Avocado
21,0.054829,0.133767,0.010036,0.183041,1.368356,0.002702,1.060314,Organic Whole Milk,Bag of Organic Bananas


- From the output above, we can see that association of two items are from the same product category (Organic). 
- A common application of association rules mining is in the domain of recommender systems. Once item pairs have been identified as having positive relationship, recommendations can be made to customers in order to increase sales. 